## Fundamental Analysis Assistant using AI
fetches 10K report from Edgar database and performs Piotroski F-Score analysis

In [2]:
# initialize environment variables
import os
import numpy as np
ZILLIZ_QNA_URI = os.getenv('ZILLIZ_QNA_URI')
ZILLIZ_USER = os.getenv('ZILLIZ_USER')
ZILLIZ_PWD = os.getenv('ZILLIZ_PWD')

'DGSp3mb2sERmp2D'

In [3]:
import warnings
warnings.filterwarnings('ignore')

from sec_edgar_downloader import Downloader

# Initialize a downloader instance. If no argument is passed
# to the constructor, the package will download filings to
# the current working directory.
dl = Downloader("./data")

# Get all 10-K filings for Apple (ticker: NVDA)
dl.get("10-K", "NVDA", after='2017-01-01' ,before='2023-01-01')

6

In [7]:
# load downloaded data 
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    './data/sec-edgar-filings/NVDA/10-K/', # my local directory
    glob='**/*.txt',     # load txt files only
    show_progress=True
)
docs = loader.load();

100%|██████████| 6/6 [05:38<00:00, 56.46s/it]


In [5]:
from pymilvus import MilvusClient
from pymilvus import utility, connections

COLLECTION_NAME = 'fundamental'  # Collection name
connections.connect(uri=ZILLIZ_QNA_URI, user=ZILLIZ_USER, 
                    password=ZILLIZ_PWD, secure=True)

# Remove the collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)


In [ ]:
from langchain.vectorstores.zilliz import Zilliz
from langchain.embeddings import HuggingFaceEmbeddings
import sentence_transformers
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0
)

docs_split = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings()
index_params = {
    'index_type': 'AUTOINDEX',
    'metric_type': 'L2'
}

doc_db = Zilliz.from_documents(docs_split, 
                               embedding,
                               collection_name = COLLECTION_NAME,
                               connection_args = {'uri': ZILLIZ_QNA_URI,
                                                  'user': ZILLIZ_USER,
                                                  'password': ZILLIZ_PWD
                                                  },
                               index_params = index_params)
